<a href="https://colab.research.google.com/github/rpathak38/SongRecommender/blob/main/Songs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import sklearn.decomposition as decomp
import matplotlib.pyplot as plt
import csv
import pickle

In [43]:
# Data Cleaning and Organization
data = pd.read_csv("/home/data.csv")
names = data[['name']]
artist = data[['artists']]
id = data[['id']]

dropThis = ['year', 'artists', 'explicit', 'duration_ms', 'popularity', 'id', 'release_date', 'name', 'key']
for name in dropThis:
    data = data.drop(name, 1);

print('done')


done


In [44]:
# Normalization (Scaling)
X = data.to_numpy()
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [45]:
# Dimensionality Reduction (PCA)
#pca = decomp.KernelPCA(n_components=2, kernel='rbf')
#X = pca.fit_transform(X)
#print(X.shape)

In [46]:
# Agglomerative Clustering
agglomerative = MiniBatchKMeans(n_clusters=16000, batch_size=32, random_state=101, verbose=1, init_size=160001)
print(agglomerative.fit(X))

# plt.scatter(X[:,0], X[:,1], c = agglomerative.labels_, cmap= 'rainbow')
# plt.show()


Init 1/3 with method: k-means++
Inertia for init 1/3: 50489.877224
Init 2/3 with method: k-means++
Inertia for init 2/3: 50323.480743
Init 3/3 with method: k-means++
Inertia for init 3/3: 50293.161212
Minibatch iteration 1/533300: mean batch inertia: 0.393552, ewa inertia: 0.393552 
Minibatch iteration 2/533300: mean batch inertia: 0.361506, ewa inertia: 0.393540 
Minibatch iteration 3/533300: mean batch inertia: 0.418968, ewa inertia: 0.393550 
Minibatch iteration 4/533300: mean batch inertia: 0.348021, ewa inertia: 0.393533 
Minibatch iteration 5/533300: mean batch inertia: 0.440149, ewa inertia: 0.393550 
Minibatch iteration 6/533300: mean batch inertia: 0.381828, ewa inertia: 0.393546 
Minibatch iteration 7/533300: mean batch inertia: 0.460157, ewa inertia: 0.393571 
Minibatch iteration 8/533300: mean batch inertia: 0.309308, ewa inertia: 0.393539 
Minibatch iteration 9/533300: mean batch inertia: 0.339562, ewa inertia: 0.393519 
[MiniBatchKMeans] Reassigning 16 cluster centers.
Mi

In [47]:
print(agglomerative.inertia_)

64187.03558209291


In [48]:
with open("/home/cluster.csv", "w") as fout:
    csv_writer = csv.writer(fout)
    temp = list(zip(names.values.flatten(), artist.values, id.values.flatten(), agglomerative.labels_))
    temp.sort(key = lambda x: x[3])
    for name, artist, id, label in temp:
        csv_writer.writerow([name, artist, id, label])

In [49]:
from google.colab import files

files.download('/home/cluster.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
from google.colab import files

files.download('/content/model_save.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
def migrate(model, scaler):
  pickle.dump(model, open("model_save.pkl", "wb"))
  pickle.dump(scaler, open("scaler_save.pkl", "wb"))

migrate(agglomerative, sc)